<a href="https://colab.research.google.com/github/Derinhelm/graph_syntax_parsing/blob/main/Parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [1]:
!pip install -q transformers

In [2]:
!pip install torch_geometric

from IPython.display import clear_output

clear_output()

In [3]:
!pip install jax



In [5]:
!pip install -U "jax[cuda12_pip]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

clear_output()

# uuparser/parser.py

In [9]:
from utils import set_seeds, read_conll
from project_logging import logging
from run import run
from embeddings import create_embeds, load_embeds

# Execution

In [24]:
real_dataset = False
embed_pickle_using = False

In [16]:
if real_dataset:
  train_a_dir = 'UD_Russian-SynTagRus/ru_syntagrus-ud-train-a.conllu'
  train_b_dir = 'UD_Russian-SynTagRus/ru_syntagrus-ud-train-b.conllu'
  train_c_dir = 'UD_Russian-SynTagRus/ru_syntagrus-ud-train-c.conllu'

  val_dir = 'UD_Russian-SynTagRus/ru_syntagrus-ud-dev.conllu'
  test_dir = 'UD_Russian-SynTagRus/ru_syntagrus-ud-test.conllu'

  train_a, train_words_a = read_conll(train_a_dir)
  train_b, train_words_b = read_conll(train_b_dir)
  train_c, train_words_c = read_conll(train_c_dir)
  train = train_a + train_b + train_c
else:
  train_dir = 'UD_Russian-SynTagRus-small/ru_syntagrus-ud-train.conllu'
  val_dir = 'UD_Russian-SynTagRus-small/ru_syntagrus-ud-dev.conllu'
  test_dir = 'UD_Russian-SynTagRus-small/ru_syntagrus-ud-test.conllu'
  
  train, train_words = read_conll(train_dir)
val, val_words = read_conll(val_dir)
test, test_words = read_conll(test_dir)

In [25]:

if embed_pickle_using:
  if real_dataset:
    embeds = load_embeds("UD_Russian-SynTagRus/embeds.pickle")
  else:
    embeds = load_embeds("UD_Russian-SynTagRus-small/embeds.pickle")
else:
  if real_dataset:
    all_words = train_words_a | train_words_b | train_words_c | val_words | test_words
  else:
    all_words = train_words | val_words | test_words
  embeds = create_embeds(all_words)


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [26]:
len(embeds)

516

In [27]:
len(train)

5

In [28]:
from pathlib import Path

print(Path('app.log').read_text())

root - WARNING - This will get logged to a file
root - WARNING - New warning
root - DEBUG - New debug
root - INFO - New info
root - WARNING - This will get logged to a file
root - WARNING - New warning
root - DEBUG - New debug
root - INFO - New info
root - WARNING - This will get logged to a file
root - WARNING - New warning
root - DEBUG - New debug
root - INFO - New info
root - WARNING - This will get logged to a file
root - WARNING - New warning
root - DEBUG - New debug
root - INFO - New info
root - WARNING - This will get logged to a file
root - WARNING - New warning
root - DEBUG - New debug
root - INFO - New info
jaxlib.mlir._mlir_libs - DEBUG - Initializing MLIR with module: _site_initialize_0
jaxlib.mlir._mlir_libs - DEBUG - Registering dialects from initializer <module 'jaxlib.mlir._mlir_libs._site_initialize_0' from '/home/derin/anaconda3/lib/python3.8/site-packages/jaxlib/mlir/_mlir_libs/_site_initialize_0.so'>
jax._src.xla_bridge - DEBUG - No jax_plugins namespace packages av

In [30]:
options = {}
options["hidden_dims"] = 100 # MLP hidden layer dimensions
options["learning_rate"] = 0.001 # Learning rate for neural network optimizer

options["dynamic_oracle"] = True # Use the static oracle instead of the dynamic oracle

options["epochs"] = 3 # Number of epochs
options["first_epoch"] = 1

# really important to do this before anything else to make experiments reproducible
set_seeds()

In [31]:
run(train, val, test, embeds, options)

RuntimeError: The expanded size of the tensor (312) must match the existing size (768) at non-singleton dimension 0.  Target sizes: [312].  Tensor sizes: [768]

In [ ]:
logging.info(f'evaluate_time: {evaluate_time:.2g}s, transform_time:{transform_time:.2g}')

In [ ]:
from google.colab import files

files.download("app.log")

# TODO

TODO:
В sentence последний элемент -

{'id': 0,
 'form': '*root*',
 'char_rep': '*root*',
 'norm': '*root*',
 'cpos': 'ROOT-CPOS',
 'pos': 'ROOT-POS',
 'parent_id': -1,
 'relation': 'rroot',
 'lemma': '*root*',
 'feats': '_',
 'deps': '_',
 'misc': '_',
 'pred_parent_id': None,
 'pred_relation': None,
 'treebank_id': None,
 'proxy_tbank': None,
 'pred_pos': None,
 'pred_cpos': None,
 'projective_order': 0,
 'rdeps': [8],
 'children': [],
 'scores': None,
 'parent': None,
 'vecs': None}


В какую сторону стек в коде сейчас ?
Используют stack[-1], stack[-2].
Стек или очередь ?

Разобраться, какие метрики считают при обучении (на train)